In [1]:
import json
import requests
import re
from pyquery import PyQuery as pq

In [2]:
with open('links.json', 'r') as input:
    links=input.read()

urls = json.loads(links)

In [3]:
dataSet=[]

In [4]:
requests.get("https://www.booking.com")
for url in urls["urls"]:
    baseUrl=url
    print(f"URL : {baseUrl}")
    page=1
    nextPage = True
    if page==1:         
        response = requests.get(baseUrl).text
        html = pq(response).find('body')
        print(f"Page {page}")
        if page == 1:          
            name = html.find('meta[itemprop="name"]').attr('content')
            total = html.find('.hotel_rank').attr('data-total-hotels')
            ranking = html.find('.hotel_rank').attr('data-ranking-as')
            reviewCity = html.find('.hotel_rank').attr('data-reviews-city')
            
            reviewScore = html.find('.review-score-badge:first').text()            
            if reviewScore is None:
                reviewScore = html.find('meta[itemprop="ratingValue"]').attr('content')
            
            reviewBasedOn = html.find('.review_list_score_count').text()
            if reviewBasedOn is None:
                reviewBasedOn = html.find('meta[itemprop="ratingCount"]').attr('content')            
            rank = f"{name} -- {ranking} out of {total} from {reviewCity} -- Review Score:{reviewScore}, {reviewBasedOn}"    
            print(rank)
            
            listScores = html.find('#review_list_score_breakdown li')  
            scores={}
            for list in listScores:
                li = pq(list)
                key = li.find('.review_score_name').text()
                if key is not None:
                    value = li.find('.review_score_value').text()
                    key = re.sub(r"\s",'_',key.lower().strip())
                    scores[key]=value  
            hotelData={'url':baseUrl,'name':name,'ranking':ranking,'total':total,'reviewCity':reviewCity,
                           'reviewScore':reviewScore,'reviewBasedOn':reviewBasedOn,'scores':scores}
        page=page+1
        reviews = html.find('ul.review_list li.review_item')
        for review in reviews:
            review = pq(review)
            published = review.find('p.review_item_date').text()
            published = re.sub(r"Reviewed:",'',published).strip() #Reviewed:
            author = review.find('.reviewer_name').text()
            country = review.find('[itemprop="nationality"] [itemprop="name"]').text()
            author_reviewcount = review.find('.review_item_user_review_count').text() 
            author_reviewcount = re.sub(r"reviews",'',author_reviewcount).strip() #reviews
            rating = review.find('.review-score-badge').text()
            if rating is None:
                rating = review.find('[itemprop="reviewRating"] meta[itemprop="ratingValue"]').attr('content')
            title = review.find('.review_item_header_content span').text()
            tags=[]
            for tag in review.find('ul.review_item_info_tags li.review_info_tag'):
                tag = pq(tag)
                tagText = re.sub(r"^(\W+)\s+",'',tag.text()) 
                tagText = re.sub(r"\&bull\;",'',tagText)
                tagText = re.sub(r"\\u2022",'',tagText)
                tagText = re.sub(r"^(\W+)\s+",'',tagText.strip()) 
                tags.append(tagText)            
            positive = review.find('.review_pos [itemprop="reviewBody"]').text()
            negative = review.find('.review_neg [itemprop="reviewBody"]').text()
            stay = review.find('.review_staydate').text()
            stay = re.sub(r"Stayed\sin",'',stay).strip()
            
            
            dataSet.append(
                {'hotel':hotelData,'reviews':{'title':title,'published':published,'author':author,'country':country,
                'author_reviewcount':author_reviewcount,'rating':rating,'tags':' | '.join(tags),'positive':positive,
                'negative':negative,'stayDate':stay}})
            

URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html
Page 1
Miller & Carter Heaton Chapel by Innkeeper's Collection -- 49 out of 110 from Manchester -- Review Score:8.3, Based on 250 hotel reviews
URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-chester-christleton.en-gb.html
Page 1
Innkeeper's Collection Chester, Christleton -- 9 out of 31 from Chester -- Review Score:8.6, Based on 1227 hotel reviews
URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-alderley-edge.en-gb.html
Page 1
The Merlin by Innkeeper's Collection -- 2 out of 5 from Alderley Edge -- Review Score:8.2, Based on 822 hotel reviews
URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-maidstone.en-gb.html
Page 1
Innkeeper's Collection Maidstone -- 3 out of 12 from Maidstone -- Review Score:8.1, Based on 879 hotel reviews
URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-solihull-knowle.en-gb.html
Page 1
The Kings Arms

In [38]:
# dataSet

In [5]:
import json

with open("reviews_first_page.json", "w") as file:
    json.dump(dataSet, file, indent=4, sort_keys=False)